In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (50, 10)


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

In [2]:
#Чтобы открыть необходимый нам датафрейм, в котором собраны все данные, в этом случае это 'objects.csv'
objects = pd.read_csv('objects.csv', sep=';')

In [3]:
#Чтобы изменить дату в столбце AgreementDate, а format это указывает программе, какой формат даты записан в objects
objects['AgreementDate'] = pd.to_datetime(objects['AgreementDate'], format='%d.%m.%Y %H:%M')

#Новая переменная df_sort_date будет иметь данные objects, которые были сортированы по столбцу AgreementDate(уже имеет тип datetime)
df_sort_date = objects.sort_values(by='AgreementDate')

In [4]:
#clear_df это новая чистая dataframe, который имеет столбцы Date, Area и Cost
clear_df = pd.read_csv('clear_df.csv', sep=';')

In [5]:
#Программа будет идти по objects, и искать необходимый нам ЖК, и которые были проданы. И анализированные данные будут в датафрейме clear_df
for i in range(len(objects)):
    if df_sort_date['ObjectName'][i] == 'Арнау - 8' and str(df_sort_date['AgreementDate'][i]) != 'NaT':
        clear_df = clear_df.append({'Date': df_sort_date['AgreementDate'][i], 'Area':df_sort_date['Area'][i], 'Cost':df_sort_date['Cost'][i]}, ignore_index=True)

#Теперь clear_df будет отсортирован по дате
clear_df = clear_df.sort_values(by='Date')

In [6]:
#Теперь отправим необходимый нам dataframe 
df = clear_df
jk_table = df[0:0]
df['Date'] = pd.to_datetime(df['Date'])
df.index = df['Date']
str(df.sort_index()['Date'][0]).split(' ')[0]
str(df.sort_index()['Date'][-1]).split(' ')[0]

'2019-06-28'

In [7]:
#определяет первый день
first_day = df['Date'][0]
#определяет второй день
last_day = df['Date'][-1]
#вычислить количество дней от начала продажи, до конца продажи
all_day = int(str(last_day - first_day).split(' ')[0])

day = datetime.strptime(str(first_day).split(' ')[0], '%Y-%m-%d')

#Этот цикл заполняет пустой jk_table с пустыми элементами
for i in range(all_day+1):
    
    jk_table = jk_table.append({'Date': str(day).split(' ')[0], 'Area':0, 'Cost':0}, ignore_index=True)
    day = (day + timedelta(days=1)).strftime('%Y-%m-%d')
    day = datetime.strptime(day, '%Y-%m-%d')

#Заново определяет дату, т.к дата сбилась после первого цикла
day = datetime.strptime(str(first_day).split(' ')[0], '%Y-%m-%d') 

#Этот цикл будет заполнять jk_table ценными информациями, которые имелись там
for j in range(len(jk_table)):
    
    area = 0
    cost = 0
    for i in range(len(df)):
        #Дает условие, если дата в заполненном с пустыми элементами jk_table с тем данным, то добавляет его.
        if jk_table['Date'][j] == str(df['Date'][i]).split(' ')[0]:
            area += float('.'.join(df['Area'][i].split(','))) #Area имеет неправильный тип
            cost += df['Cost'][i]

        
    jk_table['Date'][j] = str(day).split(' ')[0]
    jk_table['Area'][j] = area
    jk_table['Cost'][j] = cost
    day = (day + timedelta(days=1)).strftime('%Y-%m-%d')
    day = datetime.strptime(day, '%Y-%m-%d')


In [66]:
#Данная строка отправляет готовые данные в Новый csv файл
jk_table.to_csv('Proverka_Arnau8.csv', sep=';')